In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling

In [ ]:
model_name='gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#import own dataset to finetune gpt2

In [ ]:
def tokenize(batch):
  return tokenizer(batch['text'], truncation=True, padding="max_length", max_length=64)

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,   # small batch since GPT2 is heavy
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    fp16=True,  # if GPU supports mixed precision
    push_to_hub=False
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
prompt = "me aj bazaar jao ga "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


me aj bazaar jao ga ik pao chai gao bao nao ji ji ji ji ji ji ji ji ji ji ji ji ji ji ji


In [10]:
import pandas as pd

df = pd.read_csv('dataset.csv')

df.dropna()
df.drop_duplicates()

def format_qa(row):
    return f"Question: {row['question']} Answer: {row['answer']}"

df["text"] = df.apply(format_qa, axis=1)


def filter_short(example):
    return example['text'] is not None and len(example['text'].strip()) > 5

df = df[df["text"].apply(lambda x: filter_short({"text": x}))]


from sklearn.model_selection import train_test_split

train_texts, test_texts = train_test_split(df["text"], test_size=0.2, random_state=42)

dataset = {
    "train": list(train_texts),
    "test": list(test_texts)
}
from sklearn.model_selection import train_test_split

train_texts, test_texts = train_test_split(df["text"], test_size=0.2, random_state=42)

dataset = {
    "train": list(train_texts),
    "test": list(test_texts)
}

print(dataset)


from datasets import Dataset, DatasetDict

train_ds = Dataset.from_dict({"text": dataset["train"]})
test_ds = Dataset.from_dict({"text": dataset["test"]})

dataset = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

print(dataset)

{'train': ['Question: Python kis kaam aati hai? Answer: Machine learning aur data science ke liye', 'Question: Oryvo AI ka CEO kaun hai? Answer: Akhyar Ahmad', 'Question: Pakistan ka capital kya hai? Answer: Islamabad', 'Question: 2 + 2 kitna hota hai? Answer: 4'], 'test': ['Question: AI ka full form kya hai? Answer: Artificial Intelligence']}
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})
